# Create parallel coordinate plots
### Using this script you can visualize the process of bio-inspired evolutionary optimization perfromed on the tactile encoding fro reading Braille letters using the Mihilas-Niebur neuron. 

### Ref: "A Generalized Linear Integrate-and-Fire Neural Model Produces Diverse Spiking Behaviors" by Stefan Mihilas and Ernst Niebur

In [126]:
import os
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [127]:
axis_label_size = 18
axis_ticks_size = 14
dimension_label_size = axis_label_size*1
dimension_ticks_size = axis_ticks_size*1
cbar_label_size = 16
cbar_ticks_size = 14
legend_title_size = 'x-large'
legend_entry_size = 14

In [128]:
def plot_parallel_coordinate(path, experiment_number, df_plot, generations, individuals, create_pdf=False):
    # set min max values
    min_acc = np.round(np.nanmin(df_plot["fitness"]), -1)
    min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

    max_acc = np.round(np.nanmax(df_plot["fitness"]), -1)
    max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

    # fix number of ticks and set precision
    tickvals_a = np.linspace(
        np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
    tickvals_a = tickvals_a.round(0)

    tickvals_A1 = np.linspace(
        np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
    tickvals_A1 = tickvals_A1.round(2)

    tickvals_A2 = np.linspace(
        np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
    tickvals_A2 = tickvals_A2.round(2)

    # create the plot
    fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0), title={'text': f"{generations} gen., {individuals} ind.",
                                                                                                                       'y': 0.98, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'}),
                    data=go.Parcoords(
        line=dict(color=df_plot['fitness'],
                    colorscale=px.colors.diverging.Portland,
                    showscale=True,
                    cmin=0,
                    cmax=100,
                    colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                              "tickvals": np.arange(0, 100+1, 10),
                              "tickfont": dict(size=cbar_ticks_size)}),
        dimensions=list([
            dict(tickvals=tickvals_a,
                 label="a", values=df_plot['a']),
            dict(tickvals=tickvals_A1,
                 label='A1', values=df_plot['A1']),
            dict(tickvals=tickvals_A2,
                 label='A2', values=df_plot['A2']),
            dict(range=[0, 100],
                 tickvals=np.arange(0, 100+1, 20),
                 label='Accuracy (%)', values=df_plot['fitness'])
        ]),
        labelfont=dict(size=dimension_label_size),
        tickfont=dict(size=dimension_ticks_size)
    )
    )

    image_filename = f"bio_inspired_optimization_{experiment_number+1}"
    if create_pdf:
        fig.write_image(f"{path}/{image_filename}.pdf")
    fig.write_image(f"{path}/{image_filename}.png")


In [129]:
def plot_parallel_coordinate_best(path, experiment_number, df_plot, df_plot1, generations, individuals, create_pdf=False):
    # set min max values
    min_top_ten = min(df_plot1["fitness"])*1E-2
    max_top_ten = max(df_plot1["fitness"])*1E-2
    
    min_all = min(df_plot["fitness"])*1E-2
    max_all = max(df_plot["fitness"])*1E-2

    # fix number of ticks and set precision
    tickvals_a = np.linspace(np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
    tickvals_a = tickvals_a.round(0)

    tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
    tickvals_A1 = tickvals_A1.round(2)

    tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
    tickvals_A2 = tickvals_A2.round(2)
    
    fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0), title={'text': f"{generations} gen., {individuals} ind.",
                                                                                                                       'y': 0.98, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'}),
                    data=go.Parcoords(
        line=dict(color=df_plot['fitness'],
                    colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [0.96, 'orange'], [1, 'red']],
                    showscale=True,
                    cmin=min_all*1E2,
                    cmax=max_all*1E2,
                    colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                            "tickvals": np.arange(10, 100+1, 10),
                            "tickfont": dict(size=cbar_ticks_size)}),
        dimensions=list([
            dict(tickvals=tickvals_a,
                label="a", values=df_plot['a']),
            dict(tickvals=tickvals_A1,
                label='A1', values=df_plot['A1']),
            dict(tickvals=tickvals_A2,
                label='A2', values=df_plot['A2']),
            dict(range=[0, 100],
                tickvals=np.arange(0, 100+1, 20),
                label='Accuracy (%)', values=df_plot['fitness'])
        ]),
        labelfont=dict(size=dimension_label_size),
        tickfont=dict(size=dimension_ticks_size)
        )
    )

    image_filename = f"bio_inspired_optimization_{experiment_number+1}_top_ten"
    if create_pdf:
        fig.write_image(f"{path}/{image_filename}.pdf")
    fig.write_image(f"{path}/{image_filename}.png")


In [130]:
# init in and out file path
data_path = './results/'
plot_path = './plots/'

# read all files and iterate over them
all_data = os.listdir(data_path)
all_data = np.sort(all_data)

for experiment_number, data in enumerate(all_data):
    print(f"Working on {data}")
    with open(data_path + data, 'rb') as f:
        data = pickle.load(f)

    # seperate best individual and data
    best_individual = data[1::2]
    data = data[::2]

    local_data = []
    # data_best_individual = []
    for counter, individual in enumerate(data):
        local_data = np.append(local_data, individual, axis=0)
        # data_best_individual.append(individual[best_individual[counter]])
    # extrack number of individuals and generations
    print(len(data)) # nbr generations
    print(len(data[0])) # nbr individuals
    df_plot = pd.DataFrame.from_records(local_data)

    # plot all
    plot_parallel_coordinate(plot_path, experiment_number, df_plot, generations=len(data), individuals=len(data[0]), create_pdf=False)

    # plot top 10%
    df_plot1 = df_plot.iloc[:int(len(df_plot)*0.1), :]
    # df_plot2 = df_plot.iloc[int(len(df_plot)*0.1):, :]
    plot_parallel_coordinate_best(plot_path, experiment_number, df_plot, df_plot1, generations=len(data), individuals=len(data[0]), create_pdf=False)

Working on experiment_1.pkl
2
10
Working on experiment_2.pkl
34
10
Working on experiment_3.pkl
3
10
Working on experiment_4.pkl
6
25
Working on experiment_5.pkl
16
25
Working on experiment_6.pkl
2
100
Working on experiment_7.pkl
24
100
Working on experiment_8.pkl
7
100
Working on experiment_9.pkl
25
100
